# Brazilian Federal Government Budget and Payments Analysis

## Project Summary
The goal of this project is to analyze the federal expenses data related to February 2021 and connect this information to the companies registration data and also the federal budget for the year. With this data it is possible to have an overview of the destination of the federal resources.

In [1]:
import pandas as pd
import requests
import os
import zipfile

In [2]:
# Suppress cientific notation
pd.options.display.float_format = '{:,.2f}'.format

# Create dir. structure
tmp_dir = '/home/workspace/tmp/'
csv_dir = '/home/workspace/csv/'

for directory in [tmp_dir, csv_dir]: 
    if not os.path.exists(directory): 
        os.makedirs(directory)

## Project scope and data 

This project uses public data made available by the federal government. With this data, it is possible to:
1. Understand how the federal payments are distributed amoung the states
2. Understand which governmental departments use the majority of the federal resources 
3. Understand what is the main comercial activity that receive most of the federal resources (i.e., which companies receive most payments and what those companies do)

Data extracted from:
1. Federal payments data (available in zipped csv files): http://portaldatransparencia.gov.br/download-de-dados/despesas-favorecidos/
2. Companies registry data (available in zipped csv files): http://portaldatransparencia.gov.br/download-de-dados/favorecidos-pj/202102
3. Federal budget data (available in zipped csv files): http://portaldatransparencia.gov.br/download-de-dados/orcamento-despesa/2021

In [3]:
# Download data from URLs

payments_data_url = 'http://portaldatransparencia.gov.br/download-de-dados/despesas-favorecidos/202102'
companies_data_url = 'http://portaldatransparencia.gov.br/download-de-dados/favorecidos-pj/202102'
budget_data_url = 'http://portaldatransparencia.gov.br/download-de-dados/orcamento-despesa/2021'

url_zip_dict = {payments_data_url: 'payments_data.zip', 
                companies_data_url: 'companies_data.zip',
                budget_data_url: 'budget_data.zip'}

for url in url_zip_dict:
    zip_file = os.path.join(tmp_dir, url_zip_dict[url])
    print('Downloading data from ' + url + ' to ' + zip_file, end = '... ')
    r = requests.get(url)

    with open(zip_file, 'wb') as f:
        f.write(r.content)

    with zipfile.ZipFile(zip_file, 'r') as z:
        z.extractall(csv_dir)

    print('Done')

In [4]:
# Load data into pandas dataframes

for file in os.listdir(csv_dir):
    csv_file = os.path.join(csv_dir, file)
    df = pd.read_csv(csv_file, encoding = 'latin1', sep = ';', dtype = str)
    
    if 'orcamento' in csv_file.lower():
        df_budget = df
        print('Loaded Budget data into df_budget')
    elif 'cnae' in csv_file.lower():
        df_cnae = df
        print('Loaded CNAE data into df_cnae')
    elif 'cnpj' in csv_file.lower():
        df_cia = df
        print('Loaded Companies data into df_cia')
    elif 'recebimentos' in csv_file.lower():
        df_payments = df
        print('Loaded Payments data into df_payments')

Loaded Budget data into df_budget
Loaded CNAE data into df_cnae
Loaded Companies data into df_cia
Loaded Payments data into df_payments


## Data exploration and cleaning 

In [5]:
# Companies data overview

df_cia.info()   
# There are some missing data related to the companies addresses -
# as this information is not necessary, such null values are not relevant

# Remove any registration numbers that are null:
df_cia = df_cia.loc[df_cia['CNPJ'].isna() == False]

# Remove registration numbers related to foreign companies
df_cia = df_cia.loc[~ df_cia['CNPJ'].str.contains('ESTRANG')]

# Remove any registration numbers that are not unique:
df_cia = df_cia.loc[df_cia['CNPJ'].duplicated(keep = False) == False]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 938471 entries, 0 to 938470
Data columns (total 13 columns):
CNPJ               938471 non-null object
RAZAOSOCIAL        912450 non-null object
NOMEFANTASIA       584104 non-null object
COD_CNAE           938471 non-null object
COD_NATJURIDICA    938471 non-null object
TIPO_PESSOA        938471 non-null object
LOGRADOURO         912450 non-null object
NUMERO             911591 non-null object
COMPLEMENTO        384113 non-null object
CEP                0 non-null object
BAIRRO             909532 non-null object
MUNICIPIO          938471 non-null object
UF                 938471 non-null object
dtypes: object(13)
memory usage: 93.1+ MB


In [6]:
# Companies' activity data overview

df_cnae.info() 

# Remove any activity codes that are not unique
df_cnae = df_cnae.loc[df_cnae['COD_SUBCLASSE'].duplicated(keep = False) == False]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1100 entries, 0 to 1099
Data columns (total 4 columns):
COD_SECAO         1100 non-null object
DESC_SECAO        1100 non-null object
COD_SUBCLASSE     1100 non-null object
DESC_SUBCLASSE    1100 non-null object
dtypes: object(4)
memory usage: 34.5+ KB


In [7]:
# Budget data overview

df_budget.info() 

# Cast money amount column to numeric type
df_budget[['ORÇAMENTO ATUALIZADO (R$)']] = df_budget[['ORÇAMENTO ATUALIZADO (R$)']].replace(',', '.', regex = True)
df_budget['ORÇAMENTO ATUALIZADO (R$)'] = pd.to_numeric(df_budget['ORÇAMENTO ATUALIZADO (R$)'], errors = 'coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24182 entries, 0 to 24181
Data columns (total 25 columns):
EXERCÍCIO                       24182 non-null object
CÓDIGO ÓRGÃO SUPERIOR           24182 non-null object
NOME ÓRGÃO SUPERIOR             24182 non-null object
CÓDIGO ÓRGÃO SUBORDINADO        24182 non-null object
NOME ÓRGÃO SUBORDINADO          24182 non-null object
CÓDIGO UNIDADE ORÇAMENTÁRIA     24182 non-null object
NOME UNIDADE ORÇAMENTÁRIA       24182 non-null object
CÓDIGO FUNÇÃO                   24182 non-null object
NOME FUNÇÃO                     24182 non-null object
CÓDIGO SUBFUNÇÃO                24182 non-null object
NOME SUBFUNÇÃO                  24182 non-null object
CÓDIGO PROGRAMA ORÇAMENTÁRIO    24182 non-null object
NOME PROGRAMA ORÇAMENTÁRIO      24182 non-null object
CÓDIGO AÇÃO                     24182 non-null object
NOME AÇÃO                       24182 non-null object
CÓDIGO CATEGORIA ECONÔMICA      24182 non-null object
NOME CATEGORIA ECONÔMICA   

In [8]:
# Payments data overview

df_payments.info() 

# Cast money amount column to numeric type
df_payments[['Valor Recebido']] = df_payments[['Valor Recebido']].replace(',', '.', regex = True)
df_payments['Valor Recebido'] = pd.to_numeric(df_payments['Valor Recebido'], errors = 'coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655434 entries, 0 to 655433
Data columns (total 12 columns):
Código Favorecido          655434 non-null object
Nome Favorecido            655434 non-null object
Sigla UF                   655434 non-null object
Nome Município             655434 non-null object
Código Órgão Superior      655434 non-null object
Nome Órgão Superior        655434 non-null object
Código Órgão               655434 non-null object
Nome Órgão                 655434 non-null object
Código Unidade Gestora     655434 non-null object
Nome Unidade Gestora       655434 non-null object
Ano e mês do lançamento    655434 non-null object
Valor Recebido             655434 non-null object
dtypes: object(12)
memory usage: 60.0+ MB


## Data Model
The data model is a star schema with "Payments" and "Budget" as fact tables and "Companies Activities" as a dimension. 

With this structure it is posible to connect information regarding how much money was allocated in each budget, how much money was spent and which companies were the ones that received most of this money. It is possible to visualize which acitivies such companies perform and also in which state they are.

## Data Pipelines
The pipeline consists of the data download (performed previously), cleaning (also performed previously) and then a merge of CNAE data to Companies data, to generate one table named Companies Activities with all companies' information. 

In [9]:
# Companies data

# Select relevant columns only
df_cia = df_cia[['CNPJ', 'RAZAOSOCIAL','COD_CNAE', 'UF']]

df_cia.head()

,CNPJ,RAZAOSOCIAL,COD_CNAE,UF
0,00000000000191,BANCO DO BRASIL SA,6422100,DF
1,00000000000272,BANCO DO BRASIL SA,6422100,AM
2,00000000000353,BANCO DO BRASIL SA,6422100,PA
3,00000000000515,BANCO DO BRASIL SA,6421200,RJ
4,00000000000604,BANCO DO BRASIL SA,6422100,BA


In [10]:
# CNAE (companies' activities) data

# Select relevant columns only 
df_cnae = df_cnae[['COD_SUBCLASSE', 'DESC_SECAO', 'DESC_SUBCLASSE']]
df_cnae = df_cnae.rename(columns={'COD_SUBCLASSE': 'COD_CNAE'})

df_cnae.head()

,COD_CNAE,DESC_SECAO,DESC_SUBCLASSE
0,0111301,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",Cultivo de arroz
1,0111302,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",Cultivo de milho
2,0111399,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",Cultivo de outros cereais não especificados an...
3,0112101,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",Cultivo de algodão herbáceo
4,0112199,"AGRICULTURA, PECUÁRIA, PRODUÇÃO FLORESTAL, PES...",Cultivo de outras fibras de lavoura temporária...


In [11]:
# Merge Companies and CNAE data

df_cia_cnae = pd.merge(df_cia, df_cnae, how = 'left', on = 'COD_CNAE')
df_cia_cnae.head()

,CNPJ,RAZAOSOCIAL,COD_CNAE,UF,DESC_SECAO,DESC_SUBCLASSE
0,00000000000191,BANCO DO BRASIL SA,6422100,DF,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...","Bancos múltiplos, com carteira comercial"
1,00000000000272,BANCO DO BRASIL SA,6422100,AM,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...","Bancos múltiplos, com carteira comercial"
2,00000000000353,BANCO DO BRASIL SA,6422100,PA,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...","Bancos múltiplos, com carteira comercial"
3,00000000000515,BANCO DO BRASIL SA,6421200,RJ,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...",Bancos comerciais
4,00000000000604,BANCO DO BRASIL SA,6422100,BA,"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS ...","Bancos múltiplos, com carteira comercial"


In [12]:
# Payments data  

# Select relevant columns
df_payments = df_payments[['Código Favorecido',
                           'Nome Órgão Superior',  
                           'Valor Recebido']]

# Rename columns
df_payments = df_payments.rename(columns={'Código Favorecido': 'CNPJ'})

df_payments.head()

,CNPJ,Nome Órgão Superior,Valor Recebido
0,06343056000125,Ministério da Saúde,"4,396.46"
1,21310222000180,Ministério da Saúde,"9,627.98"
2,66215088000177,Ministério da Saúde,"16,030.44"
3,07406373000106,Ministério da Saúde,"5,979.66"
4,08091098000141,Ministério da Saúde,"176,970.06"


In [13]:
# Budget data

# Select relevant columns
df_budget = df_budget[['NOME ÓRGÃO SUPERIOR',
                       'NOME FUNÇÃO',
                       'NOME PROGRAMA ORÇAMENTÁRIO',
                       'ORÇAMENTO ATUALIZADO (R$)']]

df_budget.head()

,NOME ÓRGÃO SUPERIOR,NOME FUNÇÃO,NOME PROGRAMA ORÇAMENTÁRIO,ORÇAMENTO ATUALIZADO (R$)
0,Presidência da República,Segurança pública,PROGRAMA DE GESTAO E MANUTENCAO DO PODER EXECU...,"74,808.00"
1,Presidência da República,Administração,BRASIL MODERNIZA,"157,152.50"
2,Presidência da República,Administração,BRASIL MODERNIZA,"4,304.50"
3,Presidência da República,Administração,BRASIL MODERNIZA,"112,720.00"
4,Presidência da República,Segurança pública,PROGRAMA DE GESTAO E MANUTENCAO DO PODER EXECU...,"1,777,167.00"


## Data Quality Checks

In [14]:
# Verify if string to numeric conversion failed

print(f'Qty of failed numeric conversions in Payments table: {df_payments["Valor Recebido"].isna().sum()}')
print(f'Qty of failed numeric conversions in Budgets table: {df_budget["ORÇAMENTO ATUALIZADO (R$)"].isna().sum()}')

Qty of failed numeric conversions in Payments table: 0
Qty of failed numeric conversions in Budgets table: 0


In [15]:
# Verify qty of registries in Companies Activities table - expected to be > 0

print('Row count in Companies Activities table')
df_cia_cnae.count()

Row count in Companies Activities table


CNPJ              912477
RAZAOSOCIAL       912450
COD_CNAE          912477
UF                912477
DESC_SECAO        682842
DESC_SUBCLASSE    682842
dtype: int64

In [16]:
# Verify qty of registries in Payments table - expected to be > 0

print('Row count in Payments table')
df_payments.count()

Row count in Payments table


CNPJ                   655434
Nome Órgão Superior    655434
Valor Recebido         655434
dtype: int64

In [17]:
# Verify qty of registries in Budgets table - expected to be > 0

print('Row count in Budgets table')
df_budget.count()

Row count in Budgets table


NOME ÓRGÃO SUPERIOR           24182
NOME FUNÇÃO                   24182
NOME PROGRAMA ORÇAMENTÁRIO    24182
ORÇAMENTO ATUALIZADO (R$)     24182
dtype: int64

## Data dictionary 

| Table Name         | Column Name                | Field Description                                                                |
|--------------------|----------------------------|----------------------------------------------------------------------------------|
| Company Activities | CNPJ                       | Company Registry Code Number (company's unique identifier)                       |
| Company Activities | RAZAOSOCIAL                | Company Name (company's name)                                                    |
| Company Activities | COD_CNAE                   | Company Activity Code (company's main activity code)                             |
| Company Activities | UF                         | Company State Code (company's state abbreviated name)                            |
| Company Activities | DESC_SUBCLASSE             | Company Activity Name (company's main activity name)                             |
| Company Activities | DESC_SECAO                 | Company Activity Category (company's main activity category)                     |
| Payments           | CNPJ                       | Company Registry Code Number (company's unique identifier)                       |
| Payments           | Nome Órgão Superior        | Government Department Name (government's department responsible for the payment) |
| Payments           | Valor Recebido             | Payed Amount (amount of money in BRL payed to the company)                       |
| Budgets            | NOME ÓRGÃO SUPERIOR        | Government Department Name (government's department name)                        |
| Budgets            | NOME FUNÇÃO                | Government Department Category Name (government's department's category name)    |
| Budgets            | NOME PROGRAMA ORÇAMENTÁRIO | Government Program Name (government's budget program name)                       |
| Budgets            | ORÇAMENTO ATUALIZADO (R$)  | Budget Amount (amount of money in BLR allocated to a budget)                     |

## Data analysis

In [18]:
# Get the total budget amount for each governmental department

df_budget.groupby(['NOME ÓRGÃO SUPERIOR'])['ORÇAMENTO ATUALIZADO (R$)'].agg(['sum']).sort_values(by = 'sum', ascending = False).head(5)

,sum
NOME ÓRGÃO SUPERIOR,
Ministério da Economia,"3,463,337,217,680.00"
Ministério da Saúde,"193,220,320,538.00"
Ministério da Cidadania,"168,834,221,395.00"
Ministério da Educação,"148,677,319,498.00"
Ministério da Defesa,"115,749,138,315.90"


In [19]:
# Get the total budget amount for each governmental department category

df_budget.groupby(['NOME FUNÇÃO'])['ORÇAMENTO ATUALIZADO (R$)'].agg(['sum']).sort_values(by = 'sum', ascending = False).head(5)

,sum
NOME FUNÇÃO,
Encargos especiais,"2,722,182,294,853.00"
Previdência social,"786,469,937,761.00"
Saúde,"182,077,249,493.00"
Assistência social,"175,814,876,498.00"
Educação,"115,179,064,829.00"


In [20]:
# Get the total budget amount for each governmental program

df_budget.groupby(['NOME PROGRAMA ORÇAMENTÁRIO'])['ORÇAMENTO ATUALIZADO (R$)'].agg(['sum']).sort_values(by = 'sum', ascending = False).head(10)

,sum
NOME PROGRAMA ORÇAMENTÁRIO,
OPERACOES ESPECIAIS: REFINANCIAMENTO DA DIVIDA INTERNA,"1,399,235,985,553.00"
OPERACOES ESPECIAIS: SERVICO DA DIVIDA INTERNA (JUROS EAMORT,"708,548,942,136.00"
NOVA PREVIDENCIA,"683,640,617,645.00"
OPERACOES ESPECIAIS: TRANSFERENCIAS CONSTITUCIONAIS E AS DEC,"366,232,795,554.00"
PROGRAMA DE GESTAO E MANUTENCAO DO PODER EXECUTIVO,"322,588,897,736.89"
ATENCAO ESPECIALIZADA A SAUDE,"99,030,141,858.00"
INCLUSAO SOCIAL POR MEIO DO BOLSA FAMILIA E DA ARTICULACAO D,"87,047,470,135.00"
OPERACOES ESPECIAIS: OUTROS ENCARGOS ESPECIAIS,"68,103,660,251.00"
PROTECAO SOCIAL NO AMBITO DO SISTEMA UNICO DE ASSISTENCIA SO,"68,090,699,662.00"


In [21]:
# Perform a left join of payments df with companies' data df

df_payments_cia = df_payments.merge(df_cia_cnae, on = 'CNPJ', how = 'left')

In [22]:
# Get the amount payed to each state

df_payments_cia.groupby(['UF'])['Valor Recebido'].agg(['sum']).sort_values(by = 'sum', ascending = False).head(5)

,sum
UF,
DF,"27,809,798,335.72"
SP,"4,735,030,288.70"
RJ,"4,199,516,226.64"
BA,"1,668,749,933.51"
MG,"1,411,090,407.72"


In [23]:
# Get the amount payed by each governmental department

df_payments_cia.groupby(['Nome Órgão Superior'])['Valor Recebido'].agg(['sum']).sort_values(by = 'sum', ascending = False).head(5)

,sum
Nome Órgão Superior,
Ministério da Saúde,"11,277,334,659.88"
Ministério de Minas e Energia,"10,227,176,971.96"
Ministério da Educação,"10,016,422,135.91"
Ministério da Defesa,"8,141,307,461.54"
Ministério do Desenvolvimento Regional,"3,928,065,372.80"


In [24]:
# Amount payed for each economical activity category

df_payments_cia['DESC_SECAO_SUBCLASSE'] = df_payments_cia['DESC_SECAO'] + ' - ' + df_payments_cia['DESC_SUBCLASSE']
df_payments_cia.groupby(['DESC_SECAO_SUBCLASSE'])['Valor Recebido'].agg(['sum']).sort_values(by = 'sum', ascending = False).head(10)

,sum
DESC_SECAO_SUBCLASSE,
"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS - Bancos múltiplos, com carteira comercial","16,224,598,392.30"
"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS - Caixas econômicas","11,465,781,274.63"
"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL - Administração pública em geral","4,850,058,850.99"
"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS - Bancos de desenvolvimento","2,204,326,471.17"
"ATIVIDADES FINANCEIRAS, DE SEGUROS E SERVIÇOS RELACIONADOS - Bancos comerciais","1,028,812,366.46"
SAÚDE HUMANA E SERVIÇOS SOCIAIS - Outras atividades de atenção à saúde humana não especificadas anteriormente,"1,002,814,786.63"
"ADMINISTRAÇÃO PÚBLICA, DEFESA E SEGURIDADE SOCIAL - Regulação das atividades de saúde, educação, serviços culturais e outros serviços sociais","934,051,728.93"
OUTRAS ATIVIDADES DE SERVIÇOS - Organismos internacionais e outras instituições extraterritoriais,"813,251,006.33"
SAÚDE HUMANA E SERVIÇOS SOCIAIS - Atividades de apoio à gestão de saúde,"802,328,319.75"


## Conclusion

* The states that received most payments from the federal government in Feb. 2021 were:
    * Federal District (approx. 28bi BRL)
    * São Paulo State (approx. 4.7bi BRL) 
    * Rio de Janeiro State (approx. 4.2bi BRL)


* The Government Departments that performed most of 2021 February's payments were:
    * Health Department (approx. 11bi BRL)
    * Energy Department (approx. 10bi BRL) 
    * Education Department (approx. 10bi BRL)


* Which is a bit surprising, given that most of the federal budget is allocated to:
    * Economy Department (approx. 3.5tri BRL)
    * Health Department (approx. 2bi BRL) 
    * Citizenship Department (approx. 1.7bi BRL)


* More specifically, most of the federal budget is destined to the following programs:
    * Special Charges (approx. 2.7tri BRL) 
        * Out of which, 2 trillion BRL are related to Domestic Debt
    * Social Security (approx. 8bi BRL)
    * Health (approx. 2bi BRL)

* Finally, the companies' categories that received most of the federal payments in Feb. 2021 were:
    * Banks (approx. 30bi BRL) - out of which:
        * Approx. 16 billion were destined to Retail Banks
        * Approx. 11 billion were destined to Savings Banks
        * Approx. 2 billion were destined to Development Banks
        * Approx. 1 billion were destined to Commercial Banks
    * Public Administration (approx. 5bi BRL)
    * Social Services (approx. 1bi BRL)
    
    
    

## Final remarks

* The main tool used in this project was the Pandas dataframe. The datasets used here were small enough to fit in memory, so no further technologies, such as Spark, were needed to support larger files.


* Is this notebook needed to be refreshed with new data, the periodicity should be the following:
    * The budget data needs to be updated yearly, as it is not supposed to change throughout the year.
    * The payments data should be updated monthly, as new payments are made available in this cadence.
    * The companies/activities data should also be updated monthly, which is the cadence in which new information is made available in the source. 